# E6: Recursive NN

Experimental Algorithm:
- 57 LFs -> 20 randomly sampled sets of 5 -> Snorkel -> 20 LFs -> analysis
- 20 LFs -> 20 randomly sampled sets of 5 -> Snorkel -> 20 LFs -> analysis
- repeat

In [ ]:
from snorkel.labeling.model.label_model import LabelModel
from snorkel.analysis import metric_score
from Our_Monitors.CD_Monitor import Informed_LabelModel
#from Our_Monitors.CDGA_Monitor import CDGAM
from Our_Monitors.CDGA_Monitor_fast import CDGAM_fast as CDGAM
from sklearn.model_selection import train_test_split
from my_utils import predict_at_abstain_rate
import numpy as np
import logging

In [ ]:
# Parameters
num_iters = N      # N
num_subsets = J    # J
subset_size = K    # K
with_replacement = True

#abstain_target = A

sig = 0.05
policy = "new"

metrics = ["accuracy","f1","coverage"]

lf_subset = list(range(57))

In [ ]:
# Using a subset of the LFs in the analysis
L_data = np.copy(L_alarms[:,lf_subset])
Y_data = alarms_df.true_label.values

In [ ]:
# Set seed as current time to get different results
np.random.seed(int(time()))

In [ ]:
all_scores = np.zeros((5,num_iters))

for iter in range(num_iters):
    logging.info("-- Iteration " + str(iter + 1) + "--")

    # Randomly sample subsets of specified size
    try:
        subsets = np.random.choice(L_data.shape[1], size=(num_subsets,subset_size), replace=with_replacement)
    except ValueError:
        logging.error("Cannot perform sample...")
        break

    # Define a new LF per subset as a Snorkel model over the LFs in the subset
    new_L_data = np.zeros((L_data.shape[0], num_subsets))

    for i, subset in enumerate(subsets):
        l_model = LabelModel(cardinality=2, verbose=True)
        l_model.fit(L_data[:,subset], n_epochs=100, lr=0.01)
        new_L_data[:,i] = l_model.predict(L_data[:,subset])

    L_data = np.copy(new_L_data)

    # Learn dependencies between new LFs (CDGAM)
    L_train, L_dev, Y_train, Y_dev = train_test_split(L_data, Y_data, test_size=0.2, shuffle=True)
    deps = CDGAM(L_dev, k=2, sig=sig, policy=policy, verbose=False, return_more_info=False)

    # Evaluate Informed Label Model on new LFs
    il_model = Informed_LabelModel(deps, cardinality=2, verbose=True)
    il_model.fit(L_train, n_epochs=100, lr=0.01)
    scores = il_model.score(L_train, Y_train, metrics=metrics)
    
    #Y_prob = il_model.predict_proba(L_train)
    #Y_pred = predict_at_abstain_rate(Y_prob, abstain_target)
    #scores = {}
    #scores["accuracy"] = metric_score(Y_train, Y_pred, metric="accuracy")
    #Y_train_temp = Y_train[Y_pred != ABSTAIN]
    #Y_pred_temp = Y_pred[Y_pred != ABSTAIN]
    #scores["f1"] = metric_score(Y_train_temp, Y_pred_temp, metric="f1")
    #scores["abstain"] = 1.0 - metric_score(Y_train, Y_pred, metric="coverage")
    
    scores["abstain"] = 1 - scores["coverage"]
    scores["num deps (CDGAM)"] = len(deps)

    # Logging
    logging.info(scores)
    all_scores[:,iter] = list(scores.values())